## Time series classification

This script generates image that links the classification (CSV file) with original image (spatial)

#### Load the shapefile containing the area of interest and a tiff file...

In [123]:
Field <- 5
#Set a number beetwet 1 and 3: 1)NDRE, 2)GNDVI, 3)NDVI
Index <- 1

In [124]:
Directory <- "/Magistrale/CV & ML/Progetto Mancini/biocer4_s2_selection_2019_2020R1/"
Indice_p = c("NDRE", "GNDVI", "NDVI")
Indice <- Indice_p[Index]
print(Field)
print(Indice)

[1] 8
[1] "NDVI"


In [125]:
#Libraies loading
library(lubridate)
library(RStoolbox)
library(raster)
library(ggplot2)
library(rgdal)
library(reshape2)
library(png)
library(magick)
library(raster)

In [126]:
read.images <- function(data.folder, selected.band) {
    file.names <- list.files(data.folder, pattern='*.tiff$')
    file.paths <- paste(data.folder, file.names, sep="/")

 

    # creates a RasterStack objects containing all image pixels
    mylayers <- stack(file.paths, bands=selected.band)
    
    mylayers
}

In [127]:
read.shapefile <- function(shapefile, mylayers) {
    pu <- shapefile(shapefile)  
    # reproject to match crs
    pu <- spTransform(pu, proj4string(mylayers)) 

 

    pu
}

In [128]:
extract.layer.mask <- function(mylayers, p) {    
    mylayers.crop <- crop(mylayers, p)
    mylayers.mask <- mask(mylayers.crop, p)
    mylayers.mask
}

In [129]:
data.folder <-"/Magistrale/CV & ML/Progetto Mancini/biocer4_s2_selection_2019_2020R1/content/data/"
dir(data.folder)


[1] "2019-10-08.tiff" "2019-10-16.tiff" "2019-10-21.tiff" "2019-10-23.tiff"
 [5] "2019-10-26.tiff" "2019-11-07.tiff" "2020-01-11.tiff" "2020-01-24.tiff"
 [9] "2020-01-29.tiff" "2020-01-31.tiff" "2020-02-05.tiff" "2020-02-08.tiff"
[13] "2020-02-15.tiff" "2020-02-28.tiff" "2020-03-06.tiff" "2020-03-11.tiff"
[17] "2020-03-14.tiff" "2020-03-16.tiff" "2020-03-19.tiff" "2020-03-21.tiff"
[21] "2020-03-29.tiff" "2020-04-03.tiff" "2020-04-05.tiff" "2020-04-08.tiff"
[25] "2020-04-10.tiff" "2020-04-15.tiff" "2020-04-18.tiff" "2020-04-23.tiff"
[29] "2020-04-25.tiff" "2020-04-30.tiff" "2020-05-10.tiff" "2020-05-25.tiff"
[33] "2020-05-28.tiff" "2020-06-02.tiff" "2020-06-09.tiff" "2020-06-12.tiff"
[37] "2020-06-17.tiff" "2020-06-19.tiff" "2020-06-22.tiff" "2020-06-24.tiff"
[41] "2020-06-27.tiff" "2020-06-29.tiff" "2020-07-02.tiff" "2020-07-04.tiff"
[45] "2020-07-09.tiff" "2020-07-12.tiff" "2020-07-14.tiff" "2020-07-19.tiff"
[49] "2020-07-22.tiff" "2020-07-24.tiff" "2020-07-27.tiff" "2020-07-29.tiff"

In [130]:
f0 <- Directory
f1 <- "Cluster/"
f2 <- Indice
f3 <- " - Field_"
f4 <- Field
f5 <- " - K_ 3 .csv"
s0 <- "appezzamento "
s1 <- Field
s2 <- ".shp"
FPCA_title <- "_FPCA"
DTC_title <- "_DTC"
filepath_deep <- paste(f0,f1,f2,f3,f4,f5,sep="")
shpFile_name <- paste(s0,s1,s2,sep="")
plt_name_FPCA <- paste(f2,f3,f4,FPCA_title,sep="")
plt_name_DTC <- paste(f2,f3,f4,DTC_title,sep="")
print(filepath_deep)
print(shpFile_name)
print(plt_name_FPCA)
print(plt_name_DTC)

[1] "/Magistrale/CV & ML/Progetto Mancini/biocer4_s2_selection_2019_2020R1/Cluster/NDVI - Field_8 - K_ 3 .csv"
[1] "appezzamento 8.shp"
[1] "NDVI - Field_8_FPCA"
[1] "NDVI - Field_8_DTC"


In [131]:
#this is hust to take a band...
bands <- c("3")
print(bands[1])
#change me....
classificationFile <- filepath_deep
shpFile <- shpFile_name
#no more changes...
shapefiles.dir <- "/Magistrale/CV & ML/Progetto Mancini/biocer4_s2_selection_2019_2020R1/shp"
dir(shapefiles.dir)
#message(shapefiles)
shapefiles <- c(shpFile)
classification = read.csv(file = classificationFile, header = FALSE )
#message(shapefiles)
for (b_i in 1:length(bands)) {
    selected.band <- bands[b_i]
    message(selected.band)
    # read tiff images 
    mylayers <- read.images(data.folder, b_i)
    for (s_i in 1:length(shapefiles)) {
        
        shape_id <- strsplit(shapefiles[s_i], '(\\s|.shp)')[[1]][2]
        shapefile <- paste(shapefiles.dir, shapefiles[s_i], sep='/')
    
        # read shapefile 
        p <- read.shapefile(shapefile, mylayers)
        mylayers.mask <- extract.layer.mask(mylayers, p)

        # extract attribute table, result is a dataframe with columns (x, y, layer1, layer2, ...)
        mylayers.point <- as.data.frame(rasterToPoints(mylayers.mask))

        d <- dim(mylayers.point)
        d1 <- d[1] 
        d2 <- d[2]
        break;
    }
    break;
}
#visualize the output
r <- raster(mylayers.mask) 
xy<- cbind("x"=mylayers.point$x, "y"=mylayers.point$y)
v <- cellFromXY(mylayers.mask, xy)
str(v)
str(classification)
r[v] <- classification$V1

file=paste(plt_name_DTC, ".png")
png(file) 
plot(r, main=plt_name_DTC)

#plot(r, maxpixels=ncell(r))
dev.off()


[1] "3"


[1] "appezzamenti 2-14.dbf"                 
 [2] "appezzamenti 2-14.prj"                 
 [3] "appezzamenti 2-14.qpj"                 
 [4] "appezzamenti 2-14.shp"                 
 [5] "appezzamenti 2-14.shx"                 
 [6] "appezzamenti pierfrancesco fattori.qgz"
 [7] "appezzamenti1-3-13.dbf"                
 [8] "appezzamenti1-3-13.prj"                
 [9] "appezzamenti1-3-13.qpj"                
[10] "appezzamenti1-3-13.shp"                
[11] "appezzamenti1-3-13.shx"                
[12] "appezzamento 10.dbf"                   
[13] "appezzamento 10.prj"                   
[14] "appezzamento 10.qpj"                   
[15] "appezzamento 10.shp"                   
[16] "appezzamento 10.shx"                   
[17] "appezzamento 11.dbf"                   
[18] "appezzamento 11.prj"                   
[19] "appezzamento 11.qpj"                   
[20] "appezzamento 11.shp"                   
[21] "appezzamento 11.shx"                   
[22] "appezzamento 12.dbf"                   
[23] "appezzamento 12.prj"                   
[24] "appezzamento 12.qpj"                   
[25] "appezzamento 12.shp"                   
[26] "appezzamento 12.shx"                   
[27] "appezzamento 15.dbf"                   
[28] "appezzamento 15.prj"                   
[29] "appezzamento 15.qpj"                   
[30] "appezzamento 15.shp"                   
[31] "appezzamento 15.shx"                   
[32] "appezzamento 16.dbf"                   
[33] "appezzamento 16.prj"                   
[34] "appezzamento 16.qpj"                   
[35] "appezzamento 16.shp"                   
[36] "appezzamento 16.shx"                   
[37] "appezzamento 5.dbf"                    
[38] "appezzamento 5.prj"                    
[39] "appezzamento 5.qpj"                    
[40] "appezzamento 5.shp"                    
[41] "appezzamento 5.shx"                    
[42] "appezzamento 6.dbf"                    
[43] "appezzamento 6.prj"                    
[44] "appezzamento 6.qpj"                    
[45] "appezzamento 6.shp"                    
[46] "appezzamento 6.shx"                    
[47] "appezzamento 7.dbf"                    
[48] "appezzamento 7.prj"                    
[49] "appezzamento 7.qpj"                    
[50] "appezzamento 7.shp"                    
[51] "appezzamento 7.shx"                    
[52] "appezzamento 8.dbf"                    
[53] "appezzamento 8.prj"                    
[54] "appezzamento 8.qpj"                    
[55] "appezzamento 8.shp"                    
[56] "appezzamento 8.shx"                    
[57] "appezzamento 9.dbf"                    
[58] "appezzamento 9.prj"                    
[59] "appezzamento 9.qpj"                    
[60] "appezzamento 9.shp"                    
[61] "appezzamento 9.shx"

3



 num [1:220] 3 26 27 28 29 30 31 49 50 51 ...
'data.frame':	220 obs. of  1 variable:
 $ V1: int  2 2 2 2 2 2 2 2 2 1 ...


png 
  2

In [132]:
cf0 <- Directory
cf1 <- "Index_csv/"
cf2 <- Field
cf3 <- "_"
cf4 <- Indice
cf5 <- "_FPCA_indice_cluster_scores.txt"
#cf6 <- ".txt"
s0 <- "appezzamento "
s1 <- Field
s2 <- ".shp"
filepath_FPCA <- paste(cf0,cf1,cf2,cf3,tolower(cf4),cf5,sep="")
#shpFile_name <- paste(s0,s1,s2,sep="")
print(filepath_FPCA)
#print(shpFile_name)

[1] "/Magistrale/CV & ML/Progetto Mancini/biocer4_s2_selection_2019_2020R1/Index_csv/8_ndvi_FPCA_indice_cluster_scores.txt"


In [133]:
#this is hust to take a band...
bands <- c("3")
print(bands[1])
#change me....
#classificationFile <- "/Magistrale/CV & ML/Progetto Mancini/biocer4_s2_selection_2019_2020R1/Index_csv/6_gndvi_FPCA_indice_cluster_scores.txt"
classificationFile <- filepath_FPCA
shpFile <- shpFile_name
#no more changes...
shapefiles.dir <- "/Magistrale/CV & ML/Progetto Mancini/biocer4_s2_selection_2019_2020R1/shp"
dir(shapefiles.dir)
#message(shapefiles)
shapefiles <- c(shpFile)
classification = read.csv(file = classificationFile, header = FALSE )
#message(shapefiles)
for (b_i in 1:length(bands)) {
    selected.band <- bands[b_i]
    message(selected.band)
    # read tiff images 
    mylayers <- read.images(data.folder, b_i)
    for (s_i in 1:length(shapefiles)) {
        
        shape_id <- strsplit(shapefiles[s_i], '(\\s|.shp)')[[1]][2]
        shapefile <- paste(shapefiles.dir, shapefiles[s_i], sep='/')
    
        # read shapefile 
        p <- read.shapefile(shapefile, mylayers)
        mylayers.mask <- extract.layer.mask(mylayers, p)

        # extract attribute table, result is a dataframe with columns (x, y, layer1, layer2, ...)
        mylayers.point <- as.data.frame(rasterToPoints(mylayers.mask))

        d <- dim(mylayers.point)
        d1 <- d[1] 
        d2 <- d[2]
        break;
    }
    break;
}
#visualize the output
r <- raster(mylayers.mask) 
xy<- cbind("x"=mylayers.point$x, "y"=mylayers.point$y)
v <- cellFromXY(mylayers.mask, xy)
str(v)
str(classification)
r[v] <- classification$V1

file=paste(plt_name_FPCA, ".png")
print(file)
png(file) 
plot(r, main=plt_name_FPCA)




#plot(r, maxpixels=ncell(r))
dev.off()

[1] "3"


[1] "appezzamenti 2-14.dbf"                 
 [2] "appezzamenti 2-14.prj"                 
 [3] "appezzamenti 2-14.qpj"                 
 [4] "appezzamenti 2-14.shp"                 
 [5] "appezzamenti 2-14.shx"                 
 [6] "appezzamenti pierfrancesco fattori.qgz"
 [7] "appezzamenti1-3-13.dbf"                
 [8] "appezzamenti1-3-13.prj"                
 [9] "appezzamenti1-3-13.qpj"                
[10] "appezzamenti1-3-13.shp"                
[11] "appezzamenti1-3-13.shx"                
[12] "appezzamento 10.dbf"                   
[13] "appezzamento 10.prj"                   
[14] "appezzamento 10.qpj"                   
[15] "appezzamento 10.shp"                   
[16] "appezzamento 10.shx"                   
[17] "appezzamento 11.dbf"                   
[18] "appezzamento 11.prj"                   
[19] "appezzamento 11.qpj"                   
[20] "appezzamento 11.shp"                   
[21] "appezzamento 11.shx"                   
[22] "appezzamento 12.dbf"                   
[23] "appezzamento 12.prj"                   
[24] "appezzamento 12.qpj"                   
[25] "appezzamento 12.shp"                   
[26] "appezzamento 12.shx"                   
[27] "appezzamento 15.dbf"                   
[28] "appezzamento 15.prj"                   
[29] "appezzamento 15.qpj"                   
[30] "appezzamento 15.shp"                   
[31] "appezzamento 15.shx"                   
[32] "appezzamento 16.dbf"                   
[33] "appezzamento 16.prj"                   
[34] "appezzamento 16.qpj"                   
[35] "appezzamento 16.shp"                   
[36] "appezzamento 16.shx"                   
[37] "appezzamento 5.dbf"                    
[38] "appezzamento 5.prj"                    
[39] "appezzamento 5.qpj"                    
[40] "appezzamento 5.shp"                    
[41] "appezzamento 5.shx"                    
[42] "appezzamento 6.dbf"                    
[43] "appezzamento 6.prj"                    
[44] "appezzamento 6.qpj"                    
[45] "appezzamento 6.shp"                    
[46] "appezzamento 6.shx"                    
[47] "appezzamento 7.dbf"                    
[48] "appezzamento 7.prj"                    
[49] "appezzamento 7.qpj"                    
[50] "appezzamento 7.shp"                    
[51] "appezzamento 7.shx"                    
[52] "appezzamento 8.dbf"                    
[53] "appezzamento 8.prj"                    
[54] "appezzamento 8.qpj"                    
[55] "appezzamento 8.shp"                    
[56] "appezzamento 8.shx"                    
[57] "appezzamento 9.dbf"                    
[58] "appezzamento 9.prj"                    
[59] "appezzamento 9.qpj"                    
[60] "appezzamento 9.shp"                    
[61] "appezzamento 9.shx"

3



 num [1:220] 3 26 27 28 29 30 31 49 50 51 ...
'data.frame':	220 obs. of  1 variable:
 $ V1: int  2 2 2 2 2 2 2 2 1 3 ...
[1] "NDVI - Field_8_FPCA .png"


png 
  2